In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

# 加载Excel文件
file_path = 'shenzhen.xlsx'
df = pd.read_excel(file_path)

# 创建一个空的DataFrame，用于存储新增的行
new_rows = pd.DataFrame({
    'A': ['无信息', '畅行', '缓行', '拥堵', '严重拥堵'],
    'B': [0, 1, 2, 3, 4]
})

# 遍历原始DataFrame的每一列，计算每个值的出现次数
for col in df.columns:
    counts = df[col].value_counts().to_dict()  # 获取每个值的计数
    new_col = []
    for value in [0, 1, 2, 3, 4]:  # 按照指定的值顺序添加计数
        new_col.append(counts.get(value, 0))
    new_rows[col] = new_col

# 在原始DataFrame后面添加两行空白行
blank_rows = pd.DataFrame(columns=df.columns, index=range(2))
df = pd.concat([df, blank_rows], ignore_index=True)

# 将新行追加到原始DataFrame中
df = pd.concat([df, new_rows], ignore_index=True)

# 创建一个空的DataFrame，用于存储每个时刻的统计结果
stats_df = pd.DataFrame(columns=['拥挤度', '0', '1', '2', '3', '4', '拥堵段落数量', '总段落数量', '拥堵比例'])

# 遍历每一列（时刻）
for index, col in enumerate(df.columns):
    counts = df[col].value_counts().to_dict()  # 获取每个拥挤度的计数
    row = {
        '小时': index,  
        '拥挤度': col,
        '0': counts.get(0, 0),
        '1': counts.get(1, 0),
        '2': counts.get(2, 0),
        '3': counts.get(3, 0),
        '4': counts.get(4, 0)
    }
    row['拥堵段落数量'] = row['2'] + row['3'] + row['4']
    row['总段落数量'] = row['0'] + row['1'] + row['2'] + row['3'] + row['4']
    row['拥堵比例'] = (row['拥堵段落数量'] / row['总段落数量']) * 100 \
                                        if row['总段落数量'] > 0 else 0
    stats_df = pd.concat([stats_df, pd.DataFrame([row])], ignore_index=True)

# 确保所有数据都是数值类型
stats_df = stats_df.apply(pd.to_numeric, errors='coerce')

# 计算拥挤度为0的整行所有数据的平均值
avg_zero = stats_df['0'].mean()

# 标记异常值
wrong = []
for index, row in stats_df.iterrows():
    if abs(row['0'] - avg_zero) > 50:
        wrong.append(index)

# 替换异常值
for idx in wrong:
    # 获取当前行
    current_row = stats_df.iloc[idx]
    
    # 寻找非异常值行
    valid_rows = []
    for offset in [-1, 1]:
        place = idx + offset
        if 0 <= place < len(stats_df) and place not in wrong:
            valid_rows.append(stats_df.iloc[place])
    
    # 如果找到非异常值行，计算平均值并替换
    if valid_rows:
        replacement = pd.DataFrame(valid_rows).mean()
        stats_df.iloc[idx] = replacement

# 将统计结果追加到原始DataFrame中
df = pd.concat([df, stats_df], ignore_index=True)

# 保存到新的Excel文件
output_path = 'shenzhen_result.xlsx'
df.to_excel(output_path, index=False)

print(f"处理完成，结果已保存到 {output_path}")